In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
from umap import UMAP
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm
import time
import warnings
import pickle
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')

## `Needed Functions`

In [2]:
def calculate_pairwise_differences_any(data):
    """
    Calculates pairwise differences between all columns in a DataFrame and adds them as new columns.
    
    For each unique pair of columns (i,j) where i < j, creates a new column with the name
    "{col_i}{col_j}" containing the difference between column i and column j.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Input DataFrame containing numerical columns to compare
        
    Returns:
    --------
    pandas.DataFrame
        Original DataFrame with additional columns for each pairwise difference
    """
    ln = len(data.columns)
    for i in range(ln):
        for j in range(ln):
            if i < j:
                diff_name = f"{data.columns[i]}{data.columns[j]}"
                data[diff_name] = data[data.columns[i]] - data[data.columns[j]]
    return data


def kmeans_fitting(dataset, n_clusters, main_dataset, percentile_closest):
    """
    Performs K-means clustering with label propagation and outlier filtering.
    
    The function:
    1. Performs K-means clustering on the input dataset
    2. Identifies cluster centroids and propagates labels from main_dataset
    3. Filters out distant points based on percentile cutoff
    
    Parameters:
    -----------
    dataset : array-like or pandas.DataFrame
        Data to be clustered (features only)
    n_clusters : int
        Number of clusters to form
    main_dataset : pandas.DataFrame
        Original dataset containing labels in 'Hclass' column
    percentile_closest : float (0-100)
        Percentile cutoff for filtering distant points within each cluster
        
    Returns:
    --------
    tuple of (pandas.DataFrame, array-like, array-like)
        - DataFrame of cluster centroids with original features and labels
        - Features of points kept after filtering
        - Propagated labels of points kept after filtering
    """
    # 1. Perform K-means clustering
    km = KMeans(n_clusters=n_clusters)
    km_results = km.fit_transform(dataset)
    
    # Find the closest point to each cluster center (medoids)
    index = [np.argmin(km_results[km.labels_ == i][:,i]) for i in range(n_clusters)]
    
    # Extract representative points from main dataset
    data = []
    j = 0
    for i in index:
        df = main_dataset[km.labels_==j]
        data.append(df.values[i])
        j+=1
    
    Ndf = pd.DataFrame(data, columns=main_dataset.columns)
    
    # 2. Label propagation - assign cluster labels from representative points
    y_propagated = np.empty(len(dataset))
    
    for k in range(n_clusters):
        y_propagated[km.labels_ == k] = Ndf['Hclass'][k]
    
    # 3. Filter out distant points based on percentile cutoff
    X_cluster_dist = km_results[np.arange(len(dataset)), km.labels_]
    for i in range(k):
        in_cluster = (km.labels_ == i)
        cluster_dist = X_cluster_dist[in_cluster]
        cutoff_distance = np.percentile(cluster_dist, percentile_closest)
        above_cutoff = (X_cluster_dist > cutoff_distance)
        X_cluster_dist[in_cluster & above_cutoff] = -1
        
    # Identify points to keep (those not marked as -1)
    partially_propagated = (X_cluster_dist != -1)
    X_par_propagated = dataset[partially_propagated]
    y_par_propagated = y_propagated[partially_propagated]
    
    return Ndf, X_par_propagated, y_par_propagated

# FP18 Data

In [3]:
filename = 'FP18_data.fit' # Fotopoulou & Paltani (2018) data
df = Table.read(filename).to_pandas()
df = df.iloc[:,:-4]
df = pd.concat([df.iloc[:,:5], calculate_pairwise_differences_any(df.iloc[:,5:])], axis=1)

# Standardize features by removing the mean and scaling to unit variance
st = StandardScaler()
result1 = st.fit_transform(pd.concat([df.iloc[:,25:]], axis=1))
standard_df = pd.DataFrame(result1, columns=df.iloc[:,25:].columns)

N_df = pd.concat([standard_df, df['Hclass']], axis=1) # Hclass: Spectroscopic labels

# Splitting 
df_train, df_test = train_test_split(N_df, test_size=0.2, stratify = df['Hclass'] ,random_state=42)

## `Semi-Supervised Learning Classifier`

In [4]:
a = np.zeros((100, len(df_test)))

for i in range(100):
    Ndf, X_par_propagated, y_par_propagated = kmeans_fitting(df_train.iloc[:,:-1], 50, df_train, 95)
    
    rf_regressor = RandomForestClassifier()
    rf_regressor.fit(X_par_propagated, y_par_propagated)
    a[i] = rf_regressor.predict(df_test.iloc[:,:-1])

    
    print(i, f1_score(df_test['Hclass'] , a[i], average=None))

0 [0.9866406  0.98848395 0.91671733]
1 [0.98763826 0.98888738 0.91484185]
2 [0.98893229 0.98943089 0.91849148]
3 [0.98795965 0.98874729 0.91459721]
4 [0.98762215 0.98924004 0.91958257]
5 [0.98795181 0.98888286 0.91470054]
6 [0.98795181 0.98903628 0.91835482]
7 [0.98861048 0.98888588 0.91560413]
8 [0.98730055 0.98863636 0.91425046]
9 [0.98762215 0.98909731 0.91885296]
10 [0.98763826 0.98854626 0.9143898 ]
11 [0.98632812 0.98563238 0.88537049]
12 [0.98827362 0.98951215 0.92194222]
13 [0.98893949 0.98936531 0.92007322]
14 [0.98762215 0.98944662 0.92241379]
15 [0.98795181 0.98923566 0.92044064]
16 [0.98794396 0.98958333 0.92174985]
17 [0.98795181 0.98936963 0.9205379 ]
18 [0.98696219 0.98855091 0.91560413]
19 [0.98698764 0.98882492 0.91753207]
20 [0.98860306 0.98895888 0.91717418]
21 [0.98827362 0.98903034 0.91697192]
22 [0.98762215 0.9856188  0.88526646]
23 [0.98861048 0.98943376 0.92007322]
24 [0.98635478 0.98875034 0.9183922 ]
25 [0.96996805 0.98815406 0.87221869]
26 [0.98762215 0.98888

In [74]:
label_df= pd.DataFrame(a.T)
label_df['semi190'] = label_df.apply(lambda row: row.value_counts().idxmax(), axis=1)
label_df['True190'] = df_test['Hclass'].values

In [15]:
print(classification_report(label_df['True190'] , label_df['semi190'], digits=3))

              precision    recall  f1-score   support

           0      0.995     0.982     0.988      1546
           1      0.985     0.993     0.989      7353
           3      0.941     0.899     0.920       839

    accuracy                          0.983      9738
   macro avg      0.974     0.958     0.966      9738
weighted avg      0.983     0.983     0.983      9738

